# Notebook 2: Clasificación de la orientación

## 1. Estrategia para resolver la tarea:

Nuestro objetivo ahora es crear claramente una red neuronal capaz de **clasificar las imágenes según su orientación en cuatro categorías: {0°, 90°, 180°, -90°}**.

- **Entrada**: imágenes ROI (preferentemente ya recortadas a partir de la tarea anterior), posiblemente tamaño reducido (por ejemplo, 128x128 píxeles).
- **Salida**: Una clase correspondiente a la orientación del ROI (0º, 90º, 180º, -90º).

In [1]:
import pandas as pd

import os
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
def establecer_semilla(semilla=42):
    random.seed(semilla)
    np.random.seed(semilla)
    torch.manual_seed(semilla)
    torch.cuda.manual_seed_all(semilla)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Establecemos la semilla
establecer_semilla(42)

In [3]:
# Dispositivo (GPU si disponible)
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

## 2. Preparación del Dataset para clasificación

In [4]:
class OrientacionROIDataset(Dataset):
    def __init__(self, df_trazas, path_roi, image_size=(128,128)):
        self.df_trazas = df_trazas.reset_index(drop=True)
        self.path_roi = path_roi
        self.image_size = image_size
        self.label_dict = {'0':0, '90':1, '180':2, '-90':3}

    def __len__(self):
        return len(self.df_trazas)

    def __getitem__(self, idx):
        registro = self.df_trazas.iloc[idx]
        ruta_img_roi = os.path.join(self.path_roi, registro['fname'])

        img_roi = cv2.imread(ruta_img_roi, cv2.IMREAD_GRAYSCALE)
        img_roi = cv2.resize(img_roi, self.image_size) / 255.0
        img_roi = np.expand_dims(img_roi, axis=0)

        # Conversión adecuada de float a string entero
        etiqueta_orientacion = self.label_dict[str(int(float(registro['rot'])))]

        return torch.tensor(img_roi, dtype=torch.float32), etiqueta_orientacion


In [5]:
df_trazas_limpio = pd.read_csv("df_trazas_limpio.csv")
df_trazas_limpio = df_trazas_limpio.dropna(subset=["rot"]).copy()

# Separación: primero train + (val + test)
df_train, df_temp = train_test_split(
    df_trazas_limpio,
    test_size=0.3,
    stratify=df_trazas_limpio["rot"],
    random_state=42
)

# Ahora val y test (15% cada uno)
df_val, df_test = train_test_split(
    df_temp,
    test_size=0.5,
    stratify=df_temp["rot"],
    random_state=42
)

In [6]:
# Definir rutas y tamaño
path_roi = "REY_DATASET/REY_roi_rot0"
image_size = (128, 128)
batch_size = 16

# Datasets
train_dataset = OrientacionROIDataset(df_train, path_roi, image_size)
val_dataset   = OrientacionROIDataset(df_val, path_roi, image_size)
test_dataset  = OrientacionROIDataset(df_test, path_roi, image_size)

# Dataloaders
loader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
loader_val   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
loader_test  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 3. Modelo CNN adaptado a clasificación usando ResNet18

In [7]:
# ResNet18 preentrenada
resnet_clasif = resnet18(weights=ResNet18_Weights.DEFAULT)

# Ajustamos última capa para clasificación en 4 clases
resnet18.fc = nn.Sequential(
    nn.Linear(resnet_clasif.fc.in_features, 128), nn.ReLU(),
    nn.Linear(128, 4)
)

# Modelo al dispositivo
resnet_clasif = resnet_clasif.to(device)


## 4. Pérdida, optimizador y entrenamiento 

In [8]:
criterio_clasificacion = nn.CrossEntropyLoss()
optimizador = torch.optim.Adam(resnet_clasif.parameters(), lr=1e-4)

num_epocas = 20

for epoca in range(num_epocas):
    resnet_clasif.train()
    perdida_epoca_train = 0.0

    for imgs, labels in loader_train:
        imgs = imgs.repeat(1, 3, 1, 1).to(device)
        labels = labels.to(device)

        optimizador.zero_grad()
        logits = resnet_clasif(imgs)
        perdida = criterio_clasificacion(logits, labels)
        perdida.backward()
        optimizador.step()

        perdida_epoca_train += perdida.item()

    perdida_media_train = perdida_epoca_train / len(loader_train)

    # Validación
    resnet_clasif.eval()
    perdida_epoca_val = 0.0

    with torch.no_grad():
        for imgs, labels in loader_val:
            imgs = imgs.repeat(1, 3, 1, 1).to(device)
            labels = labels.to(device)

            logits = resnet_clasif(imgs)
            perdida = criterio_clasificacion(logits, labels)
            perdida_epoca_val += perdida.item()

    perdida_media_val = perdida_epoca_val / len(loader_val)

    print(f"Época [{epoca+1}/{num_epocas}] - Pérdida train: {perdida_media_train:.6f} | val: {perdida_media_val:.6f}")


Época [1/20] - Pérdida train: 2.721372 | val: 0.764039
Época [2/20] - Pérdida train: 0.202030 | val: 0.331167
Época [3/20] - Pérdida train: 0.045853 | val: 0.288785
Época [4/20] - Pérdida train: 0.019051 | val: 0.394857
Época [5/20] - Pérdida train: 0.019278 | val: 0.512999
Época [6/20] - Pérdida train: 0.012110 | val: 0.433982
Época [7/20] - Pérdida train: 0.007385 | val: 0.497065
Época [8/20] - Pérdida train: 0.004294 | val: 0.379149
Época [9/20] - Pérdida train: 0.021696 | val: 0.599762
Época [10/20] - Pérdida train: 0.024760 | val: 0.646185
Época [11/20] - Pérdida train: 0.012472 | val: 0.743102
Época [12/20] - Pérdida train: 0.005018 | val: 0.637637
Época [13/20] - Pérdida train: 0.007792 | val: 0.592858
Época [14/20] - Pérdida train: 0.016153 | val: 0.495021
Época [15/20] - Pérdida train: 0.025603 | val: 0.867058
Época [16/20] - Pérdida train: 0.022496 | val: 0.482715
Época [17/20] - Pérdida train: 0.023989 | val: 0.407945
Época [18/20] - Pérdida train: 0.016761 | val: 0.524939
É

## 5. Evaluación detallada (Accuracy y Matriz de confusión)

In [9]:
resnet_clasif.eval()
predicciones, reales = [], []

with torch.no_grad():
    for imgs, labels in loader_test:
        imgs = imgs.repeat(1, 3, 1, 1).to(device)
        labels = labels.to(device)

        logits = resnet_clasif(imgs)
        preds = logits.argmax(dim=1).cpu().numpy()
        predicciones.extend(preds)
        reales.extend(labels.cpu().numpy())

# Precisión global
precision = accuracy_score(reales, predicciones)
print(f"Accuracy global (test): {precision:.4f}")

# Matriz de confusión
matriz_confusion = confusion_matrix(reales, predicciones)
print("Matriz de confusión (test):\n", matriz_confusion)



Accuracy global (test): 0.9624
Matriz de confusión (test):
 [[45  1  0  1]
 [ 0 65  1  0]
 [ 1  0  3  0]
 [ 1  0  0 15]]


In [10]:
torch.save(resnet_clasif.state_dict(), "modelo_orientacion_resnet18.pth")